In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')

as i mentioned... that.. 
there is no directory.. of test and train...there is only text file in which there are contents.. like..>
"Akarna_Dhanurasana/16.jpg,1,8,0
Akarna_Dhanurasana/95.jpg,1,8,0
Akarna_Dhanurasana/263.jpg,1,8,0
Akarna_Dhanurasana/110.jpg,1,8,0
Akarna_Dhanurasana/21.jpg,1,8,0
Akarna_Dhanurasana/55.jpg,1,8,0
Akarna_Dhanurasana/34.jpg,1,8,0"

and the images.. are in..
/kaggle/input/82yogaclasses/yoga_images/images...


and the text files...
/kaggle/input/82yogaclasses/yoga_test.txt
/kaggle/input/82yogaclasses/yoga_train.txt


here is one more thing from where i donloaded dataset..
"This repository contains: 
1> Train and test split
	yoga_train.txt
	yoga_test.txt

Each of the split text file structure
row-> each row corresponds to each sample detail
col-> <image_address+name> , <label of class_6> , <label of class_20> , <label of class_82>

2> 82 text files correspond to each of the yoga pose with  
row-> each row corresponds to each sample detail
col-> <image_address+name> , <url>

Save the images in same <folder/image> name as given in 82 text files.


Please cite the following paper if you use this dataset.

@article{verma2020yoga,
  title={Yoga-82: A New Dataset for Fine-grained Classification of Human Poses},
  author={Verma, Manisha and Kumawat, Sudhakar and Nakashima, Yuta and Raman, Shanmuganathan},
  journal={arXiv preprint arXiv:2004.10362},
  year={2020}
}"

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

In [25]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [26]:
# Paths
data_dir = "/kaggle/input/82yogaclasses/yoga_images"  # Replace with your dataset path
yoga_train_file = "/kaggle/input/82yogaclasses/yoga_train.txt"  # Replace with actual train.txt
yoga_test_file = "/kaggle/input/82yogaclasses/yoga_test.txt"  # Replace with actual test.txt
not_found_file = "/kaggle/working/not_found_images.txt"


In [28]:
validate_images(yoga_train_file, os.path.join(data_dir, 'train'))
validate_images(yoga_test_file, os.path.join(data_dir, 'test'))


In [29]:
# Initialize class names from train.txt
def extract_classes(file_path):
    classes = set()
    with open(file_path, 'r') as f:
        for line in f:
            image_path = line.strip()
            label = os.path.basename(os.path.dirname(image_path))
            classes.add(label)
    return sorted(list(classes))

In [30]:
# Extract class names
class_names = extract_classes(yoga_train_file)

In [31]:
# Validate images function
missing_images = []
found_count = 0
missing_count = 0

In [32]:

def validate_images(file_path):
    global found_count, missing_count
    with open(file_path, 'r') as f:
        for line in f:
            image_path = line.strip()
            if not os.path.exists(image_path):
                missing_images.append(image_path)
                missing_count += 1
            else:
                found_count += 1

    # Save missing images to a file
    if missing_images:
        with open(not_found_file, 'w') as nf:
            nf.write("\n".join(missing_images))

In [33]:
def generate_file_list(data_dir, split_name, output_file):
    split_dir = os.path.join(data_dir, split_name)
    with open(output_file, "w") as f:
        for root, _, files in os.walk(split_dir):
            for file in files:
                f.write(os.path.join(root, file) + "\n")


In [34]:
generate_file_list(data_dir, "train", "/kaggle/input/82yogaclasses/yoga_train.txt")
generate_file_list(data_dir, "test", "/kaggle/input/82yogaclasses/yoga_test.txt")


OSError: [Errno 30] Read-only file system: '/kaggle/input/82yogaclasses/yoga_train.txt'

In [35]:
# Validate train and test images
print("Validating train images...")
validate_images(yoga_train_file)
print("Validating test images...")
validate_images(yoga_test_file)

Validating train images...
Validating test images...


In [36]:
with open("train.txt", "w") as train_file:
    for root, _, files in os.walk(os.path.join(data_dir, "train")):
        for file in files:
            train_file.write(os.path.relpath(os.path.join(root, file), data_dir) + "\n")


In [37]:
# Display counts
print(f"Found: {found_count}, Missing: {missing_count}")

Found: 0, Missing: 28450


In [30]:
# Debugging
if total_missing > 0:
    print("Missing files summary:")
    print("\n".join(missing_images[:10]))  # Display first 10 missing paths for inspection
    print(f"...and {total_missing - 10} more.") if total_missing > 10 else None

Missing files summary:
/kaggle/input/82yogaclasses/yoga_images/Akarna_Dhanurasana/64.jpg,1,8,0
/kaggle/input/82yogaclasses/yoga_images/Akarna_Dhanurasana/229.jpg,1,8,0
/kaggle/input/82yogaclasses/yoga_images/Akarna_Dhanurasana/128.jpg,1,8,0
/kaggle/input/82yogaclasses/yoga_images/Akarna_Dhanurasana/145.jpg,1,8,0
/kaggle/input/82yogaclasses/yoga_images/Akarna_Dhanurasana/47.jpg,1,8,0
/kaggle/input/82yogaclasses/yoga_images/Akarna_Dhanurasana/121.jpg,1,8,0
/kaggle/input/82yogaclasses/yoga_images/Akarna_Dhanurasana/100.jpg,1,8,0
/kaggle/input/82yogaclasses/yoga_images/Akarna_Dhanurasana/56.jpg,1,8,0
/kaggle/input/82yogaclasses/yoga_images/Akarna_Dhanurasana/200.jpg,1,8,0
/kaggle/input/82yogaclasses/yoga_images/Akarna_Dhanurasana/26.jpg,1,8,0
...and 28440 more.


In [31]:
# Hyperparameters
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 100
LEARNING_RATE = 0.001


In [32]:

# Classes (optional: replace with actual class names)
class_names = os.listdir(os.path.join(data_dir, 'train'))

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/82yogaclasses/yoga_images/train'

In [10]:
# Transformations for data preprocessing and augmentation
transform_train = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])


In [11]:
transform_val = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])


In [12]:
# Load Dataset
train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=transform_train)
val_dataset = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=transform_val)


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/82yogaclasses/yoga_images/train'

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# Visualize Sample Images
def show_images(loader):
    inputs, labels = next(iter(loader))
    grid = torchvision.utils.make_grid(inputs[:16], nrow=4)
    plt.figure(figsize=(10, 10))
    plt.imshow(grid.permute(1, 2, 0))  # Convert to HWC
    plt.axis("off")
    plt.show()
print("Sample Images:")
show_images(train_loader)


In [ ]:
# Models to Test
models_to_test = {
    "ResNet18": models.resnet18(pretrained=True),
    "VGG16": models.vgg16(pretrained=True),
    "EfficientNetB0": models.efficientnet_b0(pretrained=True),
    "DenseNet121": models.densenet121(pretrained=True),
}


In [ ]:
# Modify final layers for classification
for name, model in models_to_test.items():
    if "fc" in dir(model):  # For ResNet-like models
        model.fc = nn.Sequential(
            nn.Linear(model.fc.in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, len(class_names))
        )
    elif "classifier" in dir(model):  # For VGG-like models
        model.classifier[-1] = nn.Sequential(
            nn.Linear(model.classifier[-1].in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, len(class_names))
        )

In [ ]:
# Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=EPOCHS):
    best_val_acc = 0
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        model.train()
        train_loss = 0
        correct = 0
        total = 0

        for inputs, labels in tqdm(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = correct / total
        print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")

        # Validation
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        val_acc = correct / total
        print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.4f}")

        # Save the best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), f"{model.__class__.__name__}_best.pth")
            print(f"Saved Best Model: {model.__class__.__name__}")

    return best_val_acc


In [ ]:
# Train and Evaluate Models
criterion = nn.CrossEntropyLoss()
results = {}

In [ ]:
for model_name, model in models_to_test.items():
    print(f"\nTraining {model_name}...")
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    acc = train_model(model, train_loader, val_loader, criterion, optimizer, epochs=EPOCHS)
    results[model_name] = acc

In [ ]:
# Display Results
best_model_name = max(results, key=results.get)
print(f"Best Model: {best_model_name} with Accuracy: {results[best_model_name]:.4f}")

In [ ]:

# Save the Best Model
best_model = models_to_test[best_model_name].to(device)
torch.save(best_model.state_dict(), f"{best_model_name}_final.pth")
print(f"Best Model Saved as {best_model_name}_final.pth")
